# Stock Market Using BFS

In [7]:
from collections import deque

def max_profit_bfs_companies(prices):
    n = len(prices)
    companies = list(prices[0].keys())
    
    # Queue stores (day_index, holding_company_or_None, profit, transaction_history)
    queue = deque()
    queue.append((0, None, 0, []))  # start: day 0, no stock, profit 0, empty history
    max_profit = 0
    best_history = []
    
    visited = set()  # (day, holding_company, profit)
    
    while queue:
        day, holding, profit, history = queue.popleft()
        if day == n:
            if profit > max_profit:
                max_profit = profit
                best_history = history
            continue
        
        # Skip day
        state = (day + 1, holding, profit)
        if state not in visited:
            visited.add(state)
            queue.append((day + 1, holding, profit, history.copy()))
        
        # Buy a stock (if not holding any)
        if holding is None:
            for company in companies:
                new_profit = profit - prices[day][company]
                new_history = history.copy()
                new_history.append(f"Day {day}: Buy {company} at {prices[day][company]}")
                state = (day + 1, company, new_profit)
                if state not in visited:
                    visited.add(state)
                    queue.append((day + 1, company, new_profit, new_history))
        
        # Sell the stock (if holding one)
        if holding is not None:
            new_profit = profit + prices[day][holding]
            new_history = history.copy()
            new_history.append(f"Day {day}: Sell {holding} at {prices[day][holding]}")
            state = (day + 1, None, new_profit)
            if state not in visited:
                visited.add(state)
                queue.append((day + 1, None, new_profit, new_history))
    
    return max_profit, best_history

# Example usage
prices = [
    {"AAPL": 150, "GOOG": 2700, "MSFT": 300},
    {"AAPL": 155, "GOOG": 2720, "MSFT": 295},
    {"AAPL": 148, "GOOG": 2750, "MSFT": 310},
    {"AAPL": 160, "GOOG": 2800, "MSFT": 320}
]

profit, transactions = max_profit_bfs_companies(prices)
print("Maximum Profit:", profit)
print("\nTransaction Sequence:")
for t in transactions:
    print(t)


Maximum Profit: 100

Transaction Sequence:
Day 0: Buy GOOG at 2700
Day 3: Sell GOOG at 2800


# Weather Forecatsing using BFS

In [8]:
def weather_dfs(day, n, current_weather, path, all_paths, transitions):
    """
    Recursive DFS to explore weather sequences.
    
    day: current day index
    n: total days to forecast
    current_weather: current weather state
    path: current sequence of weather
    all_paths: list of all possible sequences
    transitions: dict of possible weather transitions
    """
    path.append(current_weather)
    
    if day == n - 1:
        all_paths.append(path.copy())
    else:
        for next_weather in transitions[current_weather]:
            weather_dfs(day + 1, n, next_weather, path, all_paths, transitions)
    
    path.pop()  # backtrack

# Example usage
if __name__ == "__main__":
    n_days = 3
    initial_weather = "Sunny"
    
    # Possible transitions
    transitions = {
        "Sunny": ["Sunny", "Cloudy", "Rainy"],
        "Cloudy": ["Sunny", "Cloudy", "Rainy"],
        "Rainy": ["Cloudy", "Rainy"]
    }
    
    all_sequences = []
    weather_dfs(0, n_days, initial_weather, [], all_sequences, transitions)
    
    print(f"All possible weather sequences for {n_days} days starting with {initial_weather}:")
    for seq in all_sequences:
        print(seq)


All possible weather sequences for 3 days starting with Sunny:
['Sunny', 'Sunny', 'Sunny']
['Sunny', 'Sunny', 'Cloudy']
['Sunny', 'Sunny', 'Rainy']
['Sunny', 'Cloudy', 'Sunny']
['Sunny', 'Cloudy', 'Cloudy']
['Sunny', 'Cloudy', 'Rainy']
['Sunny', 'Rainy', 'Cloudy']
['Sunny', 'Rainy', 'Rainy']


# Vechile Rotuing uinsg Hill Climbimg

In [10]:
import random

# Places
places = ["Depot", "A", "B", "C", "D"]

# Distance adjacency matrix
# matrix[i][j] = distance from place i to j
matrix = [
    [0, 2, 5, 6, 8],  # Depot
    [2, 0, 4, 7, 3],  # A
    [5, 4, 0, 2, 6],  # B
    [6, 7, 2, 0, 5],  # C
    [8, 3, 6, 5, 0]   # D
]

# Calculate total distance of a route
def total_distance(route):
    dist = 0
    for i in range(len(route) - 1):
        from_idx = places.index(route[i])
        to_idx = places.index(route[i + 1])
        dist += matrix[from_idx][to_idx]
    # Return to depot
    dist += matrix[places.index(route[-1])][places.index(route[0])]
    return dist

# Generate neighbor by swapping two locations (excluding depot)
def generate_neighbor(route):
    new_route = route.copy()
    i, j = random.sample(range(1, len(route)), 2)  # avoid depot at index 0
    new_route[i], new_route[j] = new_route[j], new_route[i]
    return new_route

# Hill Climbing Algorithm
def hill_climbing(places):
    # Initial route: start at depot, visit all other locations
    route = ["Depot"] + [p for p in places if p != "Depot"]
    random.shuffle(route[1:])
    
    current_distance = total_distance(route)
    iterations = 0
    
    while True:
        neighbor = generate_neighbor(route)
        neighbor_distance = total_distance(neighbor)
        if neighbor_distance < current_distance:
            route = neighbor
            current_distance = neighbor_distance
            iterations += 1
        else:
            break  # No better neighbor found (local optimum)
    
    return route, current_distance, iterations

# Run the hill climbing VRP
best_route, best_distance, iters = hill_climbing(places)

# Output results
print("Best route found:", best_route)
print("Total distance:", best_distance)
print("Iterations:", iters)


Best route found: ['Depot', 'A', 'D', 'C', 'B']
Total distance: 17
Iterations: 1


# Map Navigation Using A* Search

In [12]:
import heapq

# Grid map (0 = free, 1 = obstacle)
grid = [
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 1, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0]
]

start = (0, 0)
goal = (4, 4)

# Heuristic: Manhattan distance
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def astar(grid, start, goal):
    rows, cols = len(grid), len(grid[0])
    open_list = []
    heapq.heappush(open_list, (0 + heuristic(start, goal), 0, start, [start]))  # f, g, node, path
    visited = set()

    while open_list:
        f, g, current, path = heapq.heappop(open_list)

        if current in visited:
            continue
        visited.add(current)

        if current == goal:
            return path

        # Explore neighbors (up, down, left, right)
        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            nx, ny = current[0] + dx, current[1] + dy
            if 0 <= nx < rows and 0 <= ny < cols and grid[nx][ny] == 0:
                if (nx, ny) not in visited:
                    new_g = g + 1
                    new_f = new_g + heuristic((nx, ny), goal)
                    heapq.heappush(open_list, (new_f, new_g, (nx, ny), path + [(nx, ny)]))
    return None

path = astar(grid, start, goal)
print("Shortest Path:", path)

# Tic Tac Toe

Shortest Path: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4)]


# Tic Tac Toe Using Min Max

In [13]:
import math

# Board initialization
board = [' ' for _ in range(9)]  # Flattened 3x3 board

def print_board(board):
    for i in range(3):
        print(board[i*3:(i+1)*3])
    print()

def is_winner(board, player):
    win_positions = [
        [0,1,2], [3,4,5], [6,7,8],
        [0,3,6], [1,4,7], [2,5,8],
        [0,4,8], [2,4,6]
    ]
    for pos in win_positions:
        if all(board[i] == player for i in pos):
            return True
    return False

def is_draw(board):
    return ' ' not in board

def minimax(board, depth, is_maximizing):
    if is_winner(board, 'X'):
        return 1
    if is_winner(board, 'O'):
        return -1
    if is_draw(board):
        return 0

    if is_maximizing:
        best_score = -math.inf
        for i in range(9):
            if board[i] == ' ':
                board[i] = 'X'
                score = minimax(board, depth+1, False)
                board[i] = ' '
                best_score = max(score, best_score)
        return best_score
    else:
        best_score = math.inf
        for i in range(9):
            if board[i] == ' ':
                board[i] = 'O'
                score = minimax(board, depth+1, True)
                board[i] = ' '
                best_score = min(score, best_score)
        return best_score

def best_move(board, is_max_player=True):
    move = -1
    best_score = -math.inf if is_max_player else math.inf

    for i in range(9):
        if board[i] == ' ':
            board[i] = 'X' if is_max_player else 'O'
            score = minimax(board, 0, not is_max_player)
            board[i] = ' '

            if is_max_player:
                if score > best_score:
                    best_score = score
                    move = i
            else:
                if score < best_score:
                    best_score = score
                    move = i
    return move

# Main Game Loop
print("Tic Tac Toe: AI is X (Max), You are O (Min)")
while True:
    print_board(board)

    # Human O move (Min)
    o_move = int(input("Enter your move (0-8): "))
    if board[o_move] != ' ':
        print("Invalid move! Try again.")
        continue
    board[o_move] = 'O'
    print(f"Min player (O) moved to {o_move}")

    if is_winner(board, 'O'):
        print_board(board)
        print("Min player (O) wins!")
        break
    if is_draw(board):
        print_board(board)
        print("Draw!")
        break

    # AI X move (Max)
    x_move = best_move(board, True)
    board[x_move] = 'X'
    print(f"Max player (X) moved to {x_move}")

    if is_winner(board, 'X'):
        print_board(board)
        print("Max player (X) wins!")
        break
    if is_draw(board):
        print_board(board)
        print("Draw!")
        break


Tic Tac Toe: AI is X (Max), You are O (Min)
[' ', ' ', ' ']
[' ', ' ', ' ']
[' ', ' ', ' ']

Min player (O) moved to 2
Max player (X) moved to 4
[' ', ' ', 'O']
[' ', 'X', ' ']
[' ', ' ', ' ']

Min player (O) moved to 3
Max player (X) moved to 0
['X', ' ', 'O']
['O', 'X', ' ']
[' ', ' ', ' ']

Min player (O) moved to 6
Max player (X) moved to 1
['X', 'X', 'O']
['O', 'X', ' ']
['O', ' ', ' ']

Min player (O) moved to 5
Max player (X) moved to 7
['X', 'X', 'O']
['O', 'X', 'O']
['O', 'X', ' ']

Max player (X) wins!


# Optimizing Delivery Routes Using Ant colony Optimization

In [14]:
import numpy as np

def aco_tsp(distance_matrix, num_ants=10, alpha=1, beta=2, rho=0.5, iterations=100):
    n = len(distance_matrix)
    pheromone = np.ones((n, n))
    visibility = 1 / (distance_matrix + np.eye(n))  # avoid div by 0
    
    best_route = None
    best_length = float('inf')
    
    for it in range(iterations):
        routes = []
        lengths = []
        
        for ant in range(num_ants):
            unvisited = list(range(n))
            start = np.random.choice(unvisited)
            route = [start]
            unvisited.remove(start)
            
            while unvisited:
                i = route[-1]
                prob = []
                for j in unvisited:
                    tau = pheromone[i][j] ** alpha
                    eta = visibility[i][j] ** beta
                    prob.append(tau * eta)
                prob = np.array(prob) / sum(prob)
                next_city = np.random.choice(unvisited, p=prob)
                route.append(next_city)
                unvisited.remove(next_city)
            
            length = sum(distance_matrix[route[i]][route[i+1]] for i in range(n-1)) + distance_matrix[route[-1]][route[0]]
            routes.append(route)
            lengths.append(length)
            
            if length < best_length:
                best_length = length
                best_route = route
        
        # Update pheromones
        pheromone *= (1 - rho)
        for r, l in zip(routes, lengths):
            for i in range(n-1):
                pheromone[r[i]][r[i+1]] += 1 / l
            pheromone[r[-1]][r[0]] += 1 / l  # return to depot
    
    return best_route, best_length

# Example usage
distance_matrix = np.array([
    [0, 2, 9, 10],
    [1, 0, 6, 4],
    [15, 7, 0, 8],
    [6, 3, 12, 0]
])
route, length = aco_tsp(distance_matrix)
print("Best route:", route)
print("Total distance:", length)


Best route: [np.int64(3), np.int64(1), np.int64(0), np.int64(2)]
Total distance: 21


# Graph Coloring

In [1]:
def graph_coloring(graph):
    n = len(graph)
    result = [-1] * n    # color assignment for each node
    available = [False] * n  # track used colors

    # Assign color 0 to first node
    result[0] = 0

    # Assign colors to remaining nodes
    for u in range(1, n):
        # mark colors used by neighbors
        for neighbor in graph[u]:
            if result[neighbor] != -1:
                available[result[neighbor]] = True

        # find the first available color
        color = 0
        while color < n and available[color]:
            color += 1

        result[u] = color

        # reset the availability array for next iteration
        for neighbor in graph[u]:
            if result[neighbor] != -1:
                available[result[neighbor]] = False

    return result

# Example Run
graph = {
    0: [1, 2],
    1: [0, 2, 3],
    2: [0, 1, 3],
    3: [1, 2, 4, 5],
    4: [3, 5],
    5: [3, 4]
}

color_assignment = graph_coloring(graph)
print("Zone color assignment:", color_assignment)


Zone color assignment: [0, 1, 2, 0, 1, 2]


# Robot Traversal to Fetch a Tool

In [ ]:
# Robot Traversal to Fetch a Tool
# Warehouse / Workshop simulation using AI planni

class Robot:
    def __init__(self, location):
        self.location = location
        self.on_ladder = False
        self.has_tool = False

class Ladder:
    def __init__(self, location):
        self.location = location

class Tool:
    def __init__(self, location):
        self.location = location

# Actions
def move(robot, target_location):
    print(f"Robot moves from {robot.location} to {target_location}")
    robot.location = target_location

def push_ladder(robot, ladder, target_location):
    if robot.location != ladder.location:
        print("Robot cannot push ladder: not at ladder location")
        return
    print(f"Robot pushes ladder from {ladder.location} to {target_location}")
    ladder.location = target_location
    robot.location = target_location  # Robot moves with ladder

def climb_ladder(robot, ladder):
    if robot.location != ladder.location:
        print("Robot cannot climb ladder: not at ladder location")
        return
    print("Robot climbs ladder")
    robot.on_ladder = True

def descend_ladder(robot):
    if not robot.on_ladder:
        print("Robot is not on ladder")
        return
    print("Robot descends ladder")
    robot.on_ladder = False

def pick_tool(robot, ladder, tool):
    if robot.location == ladder.location == tool.location and robot.on_ladder:
        print("Robot picks up the tool")
        robot.has_tool = True
    else:
        print("Robot cannot pick tool: conditions not met")

# Planning sequence
def fetch_tool_plan():
    # Initialize objects
    robot = Robot(location='A')
    ladder = Ladder(location='B')
    tool = Tool(location='C')

    print("\n--- Initial State ---")
    print(f"Robot at {robot.location}, Ladder at {ladder.location}, Tool at {tool.location}")
    print(f"Robot on ladder? {robot.on_ladder}, Robot has tool? {robot.has_tool}\n")

    # Step 1: Move to ladder
    move(robot, ladder.location)

    
    push_ladder(robot, ladder, tool.location)

    
    climb_ladder(robot, ladder)


    pick_tool(robot, ladder, tool)

    
    descend_ladder(robot)

    print("\n--- Final State ---")
    print(f"Robot at {robot.location}, Ladder at {ladder.location}, Tool at {tool.location}")
    print(f"Robot on ladder? {robot.on_ladder}, Robot has tool? {robot.has_tool}")

    return robot.has_tool

# Run the plan
if __name__ == "__main__":
    goal_achieved = fetch_tool_plan()
    print("\nGoal achieved:", goal_achieved)



--- Initial State ---
Robot at A, Ladder at B, Tool at C
Robot on ladder? False, Robot has tool? False

Robot moves from A to B
Robot pushes ladder from B to C
Robot climbs ladder
Robot picks up the tool
Robot descends ladder

--- Final State ---
Robot at C, Ladder at C, Tool at C
Robot on ladder? False, Robot has tool? True

Goal achieved: True


# N-Queens Problem Using BackTracking

In [3]:
def is_safe(board, row, col, n):
    """
    Check if placing a queen at (row, col) is safe
    """
    # Check column
    for i in range(row):
        if board[i][col] == 1:
            return False

    # Check upper-left diagonal
    i, j = row - 1, col - 1
    while i >= 0 and j >= 0:
        if board[i][j] == 1:
            return False
        i -= 1
        j -= 1

    # Check upper-right diagonal
    i, j = row - 1, col + 1
    while i >= 0 and j < n:
        if board[i][j] == 1:
            return False
        i -= 1
        j += 1

    return True

def solve_n_queens_util(board, row, n, solutions):
    """
    Recursive utility to place queens row by row
    """
    if row == n:
        # Found a solution, add a copy to solutions
        solutions.append([''.join('Q' if c == 1 else '.' for c in r) for r in board])
        return

    for col in range(n):
        if is_safe(board, row, col, n):
            board[row][col] = 1           # Place queen
            solve_n_queens_util(board, row + 1, n, solutions)  # Recurse for next row
            board[row][col] = 0           # Backtrack

def solve_n_queens(n):
    """
    Solve the N-Queens problem and return all solutions
    """
    board = [[0 for _ in range(n)] for _ in range(n)]
    solutions = []
    solve_n_queens_util(board, 0, n, solutions)
    return solutions

# Example usage
if __name__ == "__main__":
    N = 4
    solutions = solve_n_queens(N)
    print(f"Total solutions for {N}-Queens: {len(solutions)}\n")
    for idx, sol in enumerate(solutions, 1):
        print(f"Solution {idx}:")
        for row in sol:
            print(row)
        print()


Total solutions for 4-Queens: 2

Solution 1:
.Q..
...Q
Q...
..Q.

Solution 2:
..Q.
Q...
...Q
.Q..

